# INIT

In [ ]:
# @title MOUNT DRIVE
from google.colab import drive
drive.mount('/content/drive')

import datetime

dt_now = datetime.datetime.now()

print(dt_now)

In [ ]:
# @title diffusers ver 2.4 install
%pip install torch==2.4.1 diffusers[torch]==0.25.0 huggingface_hub==0.25.2 torchvision==0.19.1 transformers==4.45.2 compel==2.0.2 omegaconf==2.4.0.dev1 pytorch-lightning accelerate safetensors==0.4.1 controlnet-aux==0.0.7 xformers==0.0.28
%apt-get install aria2

In [ ]:
# @title MODEL DL
import os
from urllib.parse import urlparse

!mkdir /content/models
!mkdir /content/vae

# @markdown token
civitai_token = "" # @param {type:"string"}

# @markdown vae
vae_url = "" # @param {type:"string"}


# @markdown ---
# @markdown model1
model1_url = "" # @param {type:"string"}

# @markdown ---
# @markdown model2
model2_url = "" # @param {type:"string"}

# @markdown ---
# @markdown model3
model3_url = "" # @param {type:"string"}

if civitai_token != "":
  token_surfix = "&token=" + civitai_token
  vae_url + token_surfix
  model1_url_dl = model1_url + token_surfix
  model2_url_dl = model2_url + token_surfix
  model3_url_dl = model3_url  + token_surfix

!aria2c -x3 --no-overwrite "{vae_url}" -d "/content/vae/"
!aria2c -x3 --no-overwrite "{model1_url_dl}" -d "/content/models/"
!aria2c -x3 --no-overwrite "{model2_url_dl}" -d "/content/models/"
!aria2c -x3 --no-overwrite "{model3_url_dl}" -d "/content/models/"


In [ ]:
# @title IMPORT
import torch
from diffusers import StableDiffusionPipeline
from diffusers import (
    DiffusionPipeline,
    DDIMScheduler,
    DDPMScheduler,
    DEISMultistepScheduler,
    DPMSolverMultistepScheduler,
    DPMSolverSinglestepScheduler,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    HeunDiscreteScheduler,
    KDPM2AncestralDiscreteScheduler,
    KDPM2DiscreteScheduler,
    UniPCMultistepScheduler,
)
from diffusers.models import AutoencoderKL
from diffusers import StableDiffusionLatentUpscalePipeline
from diffusers import StableDiffusionImg2ImgPipeline
from compel import Compel, DiffusersTextualInversionManager
from PIL import Image, ImageEnhance, ImageOps
import matplotlib.pyplot as plt
import numpy as np
import random
import time
import os
import gc


def get_latest_file(directory):
    # ディレクトリ内の全ファイルのリストを取得
    files = [os.path.join(directory, f) for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

    if not files:
        return None  # ファイルがない場合はNoneを返す

    # 最も新しいファイルを取得（最終更新時刻が最新）
    latest_file = max(files, key=os.path.getmtime)

    return latest_file


model_path = "" # @param {type:"string"}

!mkdir "./cache"

device = "cuda"

vae = AutoencoderKL.from_single_file(get_latest_file("/content/vae/"))

SCHEDULERS_LIST = {
    'DDIM' : DDIMScheduler,
    'DDPM' : DDPMScheduler,
    'DEISMultistep' : DEISMultistepScheduler,
    'DPMSolverMultistep' : DPMSolverMultistepScheduler,
    'DPMSolverSinglestep' : DPMSolverSinglestepScheduler,
    'EulerAncestralDiscrete' : EulerAncestralDiscreteScheduler,
    'EulerDiscrete' : EulerDiscreteScheduler,
    'HeunDiscrete' : HeunDiscreteScheduler,
    'KDPM2AncestralDiscrete' : KDPM2AncestralDiscreteScheduler,
    'KDPM2Discrete' : KDPM2DiscreteScheduler,
    'UniPCMultistep' : UniPCMultistepScheduler
}
####################
###### compel ######
####################

def concat_tensor(t):
    t_list = torch.split(t, 1, dim=0)
    t = torch.cat(t_list, dim=1)
    return t


def detokenize(chunk, actual_prompt):
    chunk[-1] = chunk[-1].replace('</w>', '')
    chanked_prompt = ''.join(chunk).strip()
    while '</w>' in chanked_prompt:
        if actual_prompt[chanked_prompt.find('</w>')] == ' ':
            chanked_prompt = chanked_prompt.replace('</w>', ' ', 1)
        else:
            chanked_prompt = chanked_prompt.replace('</w>', '', 1)
    actual_prompt = actual_prompt.replace(chanked_prompt,'')
    return chanked_prompt.strip(), actual_prompt.strip()

def tokenize_line(line, tokenizer): # split into chunks
    actual_prompt = line.lower().strip()
    actual_tokens = tokenizer.tokenize(actual_prompt)
    max_tokens = tokenizer.model_max_length - 2
    separators = {
        'comma': tokenizer.tokenize(',')[0],
        'dot': tokenizer.tokenize('.')[0],
        'colon': tokenizer.tokenize(':')[0]
    }

    chunks = []
    chunk = []
    for item in actual_tokens:
        chunk.append(item)
        if len(chunk) == max_tokens:
            if chunk[-1] not in list(separators.values()):
                for i in range(max_tokens-1, -1, -1):
                    if chunk[i] in list(separators.values()):
                        actual_chunk, actual_prompt = detokenize(chunk[:i+1], actual_prompt)
                        chunks.append(actual_chunk)
                        chunk = chunk[i+1:]
                        break
                else:
                    actual_chunk, actual_prompt = detokenize(chunk, actual_prompt)
                    chunks.append(actual_chunk)
                    chunk = []
            else:
                actual_chunk, actual_prompt = detokenize(chunk, actual_prompt)
                chunks.append(actual_chunk)
                chunk = []
    if chunk:
        actual_chunk, _ = detokenize(chunk, actual_prompt)
        chunks.append(actual_chunk)

    return chunks

def token_auto_concat_embeds(pipe, positive, negative):
    max_length = pipe.tokenizer.model_max_length
    positive = separate_prompt(positive)
    negative = separate_prompt(negative)
    positive_length = pipe.tokenizer(positive, return_tensors="pt").input_ids.shape[-1]
    negative_length = pipe.tokenizer(negative, return_tensors="pt").input_ids.shape[-1]

    print(f'Token length is model maximum: {max_length}, positive length: {positive_length}, negative length: {negative_length}.')
    if max_length < positive_length or max_length < negative_length:
        print('Concatenated embedding.')
        if positive_length > negative_length:
            positive_ids = pipe.tokenizer(positive, return_tensors="pt").input_ids.to("cuda")
            negative_ids = pipe.tokenizer(negative, truncation=False, padding="max_length", max_length=positive_ids.shape[-1], return_tensors="pt").input_ids.to("cuda")
        else:
            negative_ids = pipe.tokenizer(negative, return_tensors="pt").input_ids.to("cuda")
            positive_ids = pipe.tokenizer(positive, truncation=False, padding="max_length", max_length=negative_ids.shape[-1],  return_tensors="pt").input_ids.to("cuda")
    else:
        positive_ids = pipe.tokenizer(positive, truncation=False, padding="max_length", max_length=max_length,  return_tensors="pt").input_ids.to("cuda")
        negative_ids = pipe.tokenizer(negative, truncation=False, padding="max_length", max_length=max_length, return_tensors="pt").input_ids.to("cuda")

    positive_concat_embeds = []
    negative_concat_embeds = []
    for i in range(0, positive_ids.shape[-1], max_length):
        positive_concat_embeds.append(pipe.text_encoder(positive_ids[:, i: i + max_length])[0])
        negative_concat_embeds.append(pipe.text_encoder(negative_ids[:, i: i + max_length])[0])

    positive_prompt_embeds = torch.cat(positive_concat_embeds, dim=1)
    negative_prompt_embeds = torch.cat(negative_concat_embeds, dim=1)
    return positive_prompt_embeds, negative_prompt_embeds


def get_prompt_with_weight(pipe, prompt, negative):
  textual_inversion_manager = DiffusersTextualInversionManager(pipe)

  compel_proc = Compel(
      tokenizer=pipe.tokenizer,
      text_encoder=pipe.text_encoder,
      textual_inversion_manager=textual_inversion_manager,
      truncate_long_prompts=False)

  #prompt, negative = form_prompt_length(prompt, negative)

  prompt_formed = separate_prompt(prompt)
  negative_formed = separate_prompt(negative)

  print("formed prompt:")
  print(prompt_formed)
  print("formed negative:")
  print(negative_formed)

  #prompt_embeds = compel_proc([prompt_formed])
  #negative_prompt_embeds = compel_proc([negative_formed])
  prompt_embeds = compel_proc.build_conditioning_tensor(prompt_formed)
  negative_prompt_embeds = compel_proc.build_conditioning_tensor(negative_formed)

  prompt_embeds, negative_prompt_embeds = compel_proc.pad_conditioning_tensors_to_same_length([prompt_embeds, negative_prompt_embeds])

  return prompt_embeds, negative_prompt_embeds

def form_prompt_length(posi, neg):
  # posi_chunks = posi.split(",")
  # neg_chunks = neg.split(",")

  gap = len(posi) - len(neg)
  if gap > 0:
    for i in range(0,gap):
      neg += ","
  elif  gap < 0:
    for i in range(0, abs(gap)):
      posi += ","

  return posi, neg

def separate_prompt(prompt):
  #prompts = prompt.split(",")
  prompts = split_prompt(prompt)
  result = ""
  for p in prompts:
    word = replace_weight(p)
    result += word + ","
  if result[-1] == ",":
    result = result[:len(result) - 1]
  print("separate prompt:")
  print(result)
  return result

def replace_weight(word):
  pattern = r"\(.*\)"
  #s = trim_parenthesis_space(word)
  # マッチオブジェクトの取得
  matched = re.search(pattern, word)

  if matched:
    s, weight = get_word_weight(word)
    s = modify_string(s, weight)

    return s
  else:
    return word


def modify_string(s, weight):
    # 数字が1よりも大きかったら
    if weight == 0:
      return ""
    if weight >= 1:
        # 数字の数だけ+を追加
        #=(E42-1)*10+1
        num = (weight-1)*10+1
        return s + "+" * int(num)

    # 数字が1より小さかったら
    else:
        # 0.1ごとに-を追加
        num=abs(weight-1)*10+1
        #num = abs(weight)*10+1
        return s + "-" * int(num)

def get_word_weight(s):

  # 正規表現パターン
  pattern = r":([+-]?(?:\d+\.?\d*|\.\d+))"

  # マッチオブジェクトの取得
  match = re.search(pattern, s)

  # グループの取得
  try:
    result = float(match.group(1))
  except Exception as e:
    print(e)
    result = 1.0


  # 文字列からグループを削除
  s = re.sub(pattern, "", s)
  return s, result

def trim_parenthesis_space(string):
    # 正規表現パターン
    pattern = r"( *\()|(\)) *"

    # 文字列を置換
    return re.sub(pattern, "", string)

def split_prompt(prompt):
  # 正規表現を定義
  pattern = r",(?![^(]*\))"

  # 正規表現で文字列を分割
  result = re.split(pattern, prompt)

  # 結果を表示
  return result

 #########################
  ##### control net #####
##########################


from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from controlnet_aux import OpenposeDetector
from controlnet_aux import HEDdetector
from controlnet_aux import NormalBaeDetector
from controlnet_aux import OpenposeDetector
from controlnet_aux import PidiNetDetector
from diffusers.utils import load_image
import cv2

!mkdir /content/contolnet_tag/
!mkdir /content/i2i_tag/

def get_latest_file_path(folder_path):
  """フォルダ内の最も最近作成されたファイルのパスを取得する関数

  Args:
    folder_path: フォルダのパスを指定する

  Returns:
    フォルダ内の最も最近作成されたファイルのパスを返す
  """

  # フォルダ内のファイル一覧を取得する
  file_list = os.listdir(folder_path)

  # ファイル一覧から、作成日時が最も新しいファイルを取得する
  latest_file = max(file_list, key=lambda file: os.stat(os.path.join(folder_path, file)).st_ctime)

  # ファイルのパスを返す
  return os.path.join(folder_path, latest_file)

def load_image_from_file(file_path):
  """画像ファイルをcv2に読み込む関数

  Args:
    file_path: 画像ファイルのパスを指定する

  Returns:
    cv2.imread()で読み込んだ画像データを返す
  """

  # ファイルの拡張子を取得する
  file_extension = os.path.splitext(file_path)[1]

  # 画像ファイルの拡張子かどうかを判別する
  if file_extension in [".jpg", ".jpeg", ".png", ".bmp"]:
    # 画像ファイルをcv2に読み込む
    return cv2.imread(file_path)
  else:
    return None

def get_control_target():
  control_img_path = get_latest_file_path("/content/contolnet_tag/")
  img = load_image_from_file(control_img_path)
  return img

def get_i2i_target():
  control_img_path = get_latest_file_path("/content/i2i_tag/")
  img = load_image_from_file(control_img_path)
  return img

def controlnet_canny():
  global model_dir_path
  tag = get_control_target()
  low_threshold = 100
  high_threshold = 200
  image = cv2.Canny(tag, low_threshold, high_threshold)
  image = image[:, :, None]
  image = np.concatenate([image, image, image], axis=2)
  canny_image = Image.fromarray(image)

  return canny_image

def resize_image(img, width, height):
  """
  画像をリサイズする関数

  Args:
    input_file: 入力画像ファイル
    output_file: 出力画像ファイル
    width: リサイズ後の幅
    height: リサイズ後の高さ

  Returns:
    None
  """

  # リサイズする

  img = img.resize((width, height), Image.LANCZOS)

  # 画像を出力する
  return img

def controlne_normal(image):
  depth_estimator = pipeline("depth-estimation", model ="Intel/dpt-hybrid-midas" )

  image = depth_estimator(image)['predicted_depth'][0]

  image = image.numpy()

  image_depth = image.copy()
  image_depth -= np.min(image_depth)
  image_depth /= np.max(image_depth)

  bg_threhold = 0.4

  x = cv2.Sobel(image, cv2.CV_32F, 1, 0, ksize=3)
  x[image_depth < bg_threhold] = 0

  y = cv2.Sobel(image, cv2.CV_32F, 0, 1, ksize=3)
  y[image_depth < bg_threhold] = 0

  z = np.ones_like(x) * np.pi * 2.0

  image = np.stack([x, y, z], axis=2)
  image /= np.sum(image ** 2.0, axis=2, keepdims=True) ** 0.5
  image = (image * 127.5 + 127.5).clip(0, 255).astype(np.uint8)
  image = Image.fromarray(image)
  return image

  ########################
  ###### load rola ######
  ########################


# 1,loraフォルダからぜｎLora を読み込んでおく
# prompt からloraをひろってもとから消す
# filenameとウェイトを読んでfuse_loraに渡す
import re
from pathlib import Path
from safetensors.torch import load_file

class lora_loader:
  lora_dir = ""
  pipe = None
  LORA_DICT = {}
  def __init__(self, pipe, fuse_loras):
    self.lora_dir = "/content/drive/MyDrive/models/Lora/" # @param {type:"string"}
    self.pipe = pipe
    self.fuse_loras = fuse_loras
    self.LORA_DICT = self.generate_filename_path_dict(self.find_files_as_dict(self.lora_dir, "safetensors"))
    print(fuse_loras)

  def find_files_as_dict(self,dir, ext):
    p = Path(dir)
    l = list(p.glob(f"**/*.{ext}"))
    return l

  def generate_filename_path_dict(self,filepaths):
    filename_path_dict = {}
    for filepath in filepaths:
      filename = os.path.splitext(os.path.basename(filepath))[0]
      filename_path_dict[filename] = filepath
    print(f"{len(filename_path_dict)} loras found.")
    return filename_path_dict

  def get_lora(self, name):
    try:
      return self.LORA_DICT[name]
    except Exception as e:
      return None

  def set_loras(self):
    name_list = []
    scale_list = []
    formed_fuse_loras = self.merge_values(self.fuse_loras)
    self.pipe.set_adapters(name_list,adapter_weights=scale_list)
    for lora in formed_fuse_loras:
      lora_name = lora[0]
      lora_path = self.get_lora(lora_name)
      scale = lora[1]
      if type(scale) is not float:
        scale = float(1)
        print(f"{lora_name}'s scale is not float. set 1.")
      if lora_path == None:
        print(f"{lora_name} is None. pass")
        write_log(lora_name+" is not exists.")
        continue
      elif os.path.exists(lora_path):
        print(f"loading{lora_path}")
      lora_dir = os.path.dirname(lora_path)
      lora_file = os.path.basename(lora_path)
      adapt_name = lora_name.replace(".","_")
      try:
        self.pipe.load_lora_weights(lora_dir, weight_name=lora_file, lora_scale=scale, low_cpu_mem_usage=False, adapter_name=adapt_name)
      except Exception as e:
        print(e)
        print(f"error occured in {lora_name}, continue")
        write_log(lora_name + "" + str(e))
        continue
      name_list.append(adapt_name)
      scale_list.append(scale)
    self.pipe.set_adapters(name_list,adapter_weights=scale_list)
    self.pipe.fuse_lora()
    self.pipe.unload_lora_weights()
    #self.pipe.unet.to(memory_format=torch.channels_last)
    #self.pipe.unet = torch.compile(self.pipe.unet, mode="reduce-overhead", fullgraph=True)

  def load_lora(self,lora_name, scale):
    lora_path = self.get_lora(lora_name)
    if type(scale) is not float:
      scale = float(1)
      print(f"{lora_name}'s scale is not float. set 1.")
    if lora_path == None:
      print(f"{lora_name} is None.")
      write_log(lora_name+" is not exists.")
    elif os.path.exists(lora_path):
      print(f"loading{lora_path}")
      try:
        #self.pipe.load_lora_weights(lora_path)
        #dir_path, file_nameをあたえる
        lora_dir = os.path.dirname(lora_path)
        lora_file = os.path.basename(lora_path)
        self.pipe.load_lora_weights(lora_dir, weight_name=lora_file, lora_scale=scale, low_cpu_mem_usage=False)
        #self.pipe.load_lora_weights(".", weight_name=lora_path, lora_scale =scale)
        # load_safetensors_lora(self.pipe, lora_path) layer syuturyoku nomi
        self.pipe.fuse_lora(lora_scale=scale)
        print(f"{lora_name} fused.")
      except Exception as e:
        print(f"error in loading {lora_name}, may be not lora.")
        print(e)
    else:
       print(f"probably {lora_name} is not exists.")

  def fusing_loras(self):
    self.set_loras()
    #2024-11-13 Loraの発言不全にともなってLora適応方法をfuseからadaptにかえる
    #for lora in self.fuse_loras:
    #  self.load_lora(lora[0], lora[1])

  def merge_values(self,a_list):
      result = {}  # 結果を格納するための空の辞書
      for item in a_list:
          key = item[0]    # 0番目の要素をキーに
          value = item[1]  # 1番目の要素を値として取得
          if key in result:
              result[key] += value  # 既にキーが存在すれば値を足し合わせる
          else:
              result[key] = value  # キーが存在しなければ新しく追加
      # 結果をリストに変換
      return [[key, value] for key, value in result.items()]


def get_loras_from_prompt(prompt):
  """
  文字列から<lora:[^>]+>に該当する文字列を抽出する

  Args:
    text: 抽出対象の文字列

  Returns:
    抽出された文字列
  """

  pattern = re.compile(r"<lora:[^>]+>")
  match = pattern.findall(prompt)
  print("get loras from prompt")
  result = []
  for mat in match:
    try:
      splitted = mat.replace("<","")
      splitted = splitted.replace(">","")
      splitted = splitted.split(":")
      if len(splitted) < 3:
        print(f"{mat}, failed to split.")
      else:
        print(f"{splitted}, ")
        name = splitted[1]
        weight = float(splitted[2])
        result.append([name,weight])
    except Exception as e:
      print(e)
  return result


def remove_lora(text):
  removed_prompt = re.sub(r"<lora:[^>]+>", "", text)
  print("removed prompt:")
  print(removed_prompt)
  return removed_prompt


# load_textual_inversion

def set_embs(prompt, pipe):
  emb_paths = fined_emb_from_dir("/content/drive/MyDrive/models/textual_inversion")
  emb_dict = make_emb_dict(emb_paths)
  splitted_prompt = split_and_strip_list_comprehension(prompt)
  for word in splitted_prompt:
    if word in emb_dict:
      # 存在する場合は、キーに対応する値を返す
      if os.path.exists(emb_dict[word]):
        pipe.load_textual_inversion(emb_dict[word], token=word)
        print(f"{word} is texture. loaded.")

def fined_emb_from_dir(dir):
  p = Path(dir)
  l = list(p.glob(f"**/*.pt"))
  return l

def make_emb_dict(filepaths):
  filename_path_dict = {}
  for filepath in filepaths:
    filename = os.path.splitext(os.path.basename(filepath))[0]
    filename_path_dict[filename] = filepath
  print(f"{len(filename_path_dict)} embds found.")
  return filename_path_dict

def split_and_strip_list_comprehension(string):

  # 文字列を","で区切って、各文字列から前後のスペースを取り除く
  return [s.strip() for s in string.split(",")]

def load_safetensors_lora(pipeline, checkpoint_path, LORA_PREFIX_UNET="lora_unet", LORA_PREFIX_TEXT_ENCODER="lora_te", alpha=0.75):
    # load LoRA weight from .safetensors
    state_dict = load_file(checkpoint_path)

    visited = []
    print(f"checkpoint_path: {checkpoint_path}, lora test")
    # directly update weight in diffusers model
    for key in state_dict:
        print(f"lora key: {key}")
        # it is suggested to print out the key, it usually will be something like below
        # "lora_te_text_model_encoder_layers_0_self_attn_k_proj.lora_down.weight"
        continue
        # as we have set the alpha beforehand, so just skip
        if ".alpha" in key or key in visited:
            continue

        if "text" in key:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_TEXT_ENCODER + "_")[-1].split("_")
            curr_layer = pipeline.text_encoder
        else:
            layer_infos = key.split(".")[0].split(LORA_PREFIX_UNET + "_")[-1].split("_")
            curr_layer = pipeline.unet

        # find the target layer
        temp_name = layer_infos.pop(0)
        while len(layer_infos) > -1:
            try:
                curr_layer = curr_layer.__getattr__(temp_name)
                if len(layer_infos) > 0:
                    temp_name = layer_infos.pop(0)
                elif len(layer_infos) == 0:
                    break
            except Exception:
                if len(temp_name) > 0:
                    temp_name += "_" + layer_infos.pop(0)
                else:
                    temp_name = layer_infos.pop(0)

        pair_keys = []
        if "lora_down" in key:
            pair_keys.append(key.replace("lora_down", "lora_up"))
            pair_keys.append(key)
        else:
            pair_keys.append(key)
            pair_keys.append(key.replace("lora_up", "lora_down"))

        # update weight
        if len(state_dict[pair_keys[0]].shape) == 4:
            weight_up = state_dict[pair_keys[0]].squeeze(3).squeeze(2).to(torch.float32)
            weight_down = state_dict[pair_keys[1]].squeeze(3).squeeze(2).to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down).unsqueeze(2).unsqueeze(3)
        else:
            weight_up = state_dict[pair_keys[0]].to(torch.float32)
            weight_down = state_dict[pair_keys[1]].to(torch.float32)
            curr_layer.weight.data += alpha * torch.mm(weight_up, weight_down)

        # update visited list
        for item in pair_keys:
            visited.append(item)

    return pipeline

def write_log(message, filename="log.txt", mode="a"):
    with open(filename, mode) as f:
        f.write(f"{message}\n")

########################
###### meta data ######
########################

# @title metadata
from PIL import PngImagePlugin

def add_metadata(img, metadata: dict):
    imgInfo = img.info
    imgInfo.update(metadata)
    info = PngImagePlugin.PngInfo()
    for k, v in imgInfo.items():
      print(k,v)
      info.add_itxt(k, str(v))
    return info



def init_metadata(input_path: str, output_path: str, metadata: dict):
    img = PngImagePlugin.PngImageFile(input_path)
    info = PngImagePlugin.PngInfo()
    for k, v in metadata.items():
        info.add_itxt(k, v)
    img.save(output_path, pnginfo=info)


def get_metadata(path: str):
    img = PngImagePlugin.PngImageFile(path)
    info = img.info
    for key, value in info.items():
        print(f'{key}: {value}')
    return info

def save_meta(img, save_path, prompt="", nega="", step="", Sampler="", CFG_scale="", seed="", size="", Model_hash="", Model="", strength="", Mask_blur=""):
    metadata = {}
    metadata["parameters"] = prompt
    metadata["Negative prompt"] = nega

    metadata["Step"] = step
    metadata["Sampler"] = Sampler
    metadata["CFG scale"] = CFG_scale
    metadata["Seed"] = seed
    metadata["Size"] = size
    metadata["Model hash"] = Model_hash
    metadata["Model"] = Model
    metadata["Denoising strength"] = strength
    metadata["Mask blur"] = Mask_blur

    info = add_metadata(img, metadata)
    img.save(save_path, pnginfo=info)

###########################
###### image adjust ######
###########################

# @title Image Adjust
import cv2
import numpy as np


def image_adjust(img, gamma=2):
  img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)  # BGRからHSVに変換
  h, s, v = cv2.split(img_hsv)  # チャンネルごとに分割

  # ルックアップテーブルの生成
  look_up_table = np.zeros((256,1),dtype=np.uint8)
  for i in range(256):
      look_up_table[i][0] = (i/255)**(1.0/gamma)*255

  s_lut = cv2.LUT(s, look_up_table)  # 彩度(S)に対してルックアップテーブル適用
  img_merge = cv2.merge([h, s_lut, v])  #  H,変換後S,Vをマージ
  img_bgr = cv2.cvtColor(img_merge, cv2.COLOR_HSV2BGR)  # HSVからBGRに変換

  return img_bgr

def manual_adjust(img, sat, con):
  if sat > 0:
    base_image = add_saturation(img, sat)
  if con > 0:
    base_image = add_contrast(img, con)

  return base_image

def adjust_contract(img, factor=1):
  img = img.convert("YCbCr")
  yy, cb, cr = img.split()

  yy = ImageOps.equalize(yy)
  yy * factor
  img = Image.merge("YCbCr", (yy, cb, cr))
  img = img.convert("RGB")
  return img

def get_saturation(img):
    a
    # 画像の幅と高さを取得
    width, height = img.size

    # 画像のピクセルデータを取得
    pixels = img.load()

    # 各ピクセルの彩度をリストに格納
    saturation_list = []
    for y in range(height):
        for x in range(width):
            # RGB値を取得
            r, g, b = pixels[x, y]

            # 彩度を計算
            saturation = (max(r, g, b) - min(r, g, b)) / 3

            # 彩度リストに追加
            saturation_list.append(saturation)
            average_saturation = sum(saturation_list) / len(saturation_list)

            # 結果を出力
            return average_saturation

def adjust_saturation(img, tag_sat, current_sat):
    enhancer = ImageEnhance.Color(img)
    saturation_factor = tag_sat / current_sat
    print(f"saturation_factor: {str(saturation_factor)}")
    enhanced_image = enhancer.enhance(saturation_factor)
    return enhanced_image

def    auto_enhance_image(image):
    """
    Pillowを使って彩度やコントラストを自動で調整し、鮮明にする処理

    Args:
        image_path:    画像ファイルパス

    Returns:
        鮮明化された画像
    """

    # 画像を開く

    # RGBチャンネルを取得
    r, g, b = image.split()

    #    ヒストグラム分析
    hist_r = np.histogram(r, 256)[0]
    hist_g = np.histogram(g, 256)[0]
    hist_b = np.histogram(b, 256)[0]

    # Otsu's methodで閾値を計算
    threshold_r = OtsuThreshold(hist_r)
    threshold_g = OtsuThreshold(hist_g)
    threshold_b = OtsuThreshold(hist_b)

    # 閾値に基づいて彩度とコントラストを調整
    enhancer_r = ImageEnhance.Contrast(r)
    enhancer_g = ImageEnhance.Contrast(g)
    enhancer_b = ImageEnhance.Contrast(b)

    factor_r = 1.0 + (1.2 - 1.0) / (255 - threshold_r)
    factor_g = 1.0 + (1.2 - 1.0) / (255 - threshold_g)
    factor_b = 1.0 + (1.2 - 1.0) / (255 - threshold_b)

    r = enhancer_r.enhance(factor_r)
    g = enhancer_g.enhance(factor_g)
    b = enhancer_b.enhance(factor_b)

    # 合成して保存
    image = Image.merge("RGB", (r, g, b))

    return image

#    Otsu's method
def OtsuThreshold(hist):
    total = hist.sum()
    sumB = 0
    wB = 0
    varMax = 0
    threshold = 0

    for i in range(0, 256):
        wB += hist[i]
        if wB == 0:
            continue

        wF = total - wB
        sumB += i * hist[i]

        mB = sumB / wB
        mF = (total * sumB - sumB) / wF

        varBetween = wB * wF * (mB - mF) ** 2

        if varBetween >= varMax:
            threshold = i
            varMax = varBetween

    return threshold

def add_saturation(img, factor):
  enhancer = ImageEnhance.Color(img)  # ImageEnhance.Colorオブジェクト生成

  factor = [factor,factor,factor]  # enhancement factorのリスト

  for i in factor:
      image_c = enhancer.enhance(i)

  return image_c

def add_contrast(img, factor):
  image = ImageEnhance.Contrast(img)
  image = image.enhance(factor)
  return image


# GENERATION

In [ ]:
# @title Generate Random_portrait generator

#test prompt

import json
import os
import random

prompt = "" # @param {type:"string"}
add_prompt = "" # @param {type:"string"}
prompt = prompt + ", " + add_prompt
neg = "" # @param {type:"string"}
add_neg = "" # @param {type:"string"}
neg = add_neg + "," + neg

# @markdown ---
# @markdown Randomizer
is_random_hair = False # @param {type:"boolean"}
is_random_face = False # @param {type:"boolean"}
gender = "none" # @param ["none","female","young_female","male","young_male","anime_girl","anime_boy"]
fix_hair_color = "" # @param {type:"string"}
fix_eyes_color = "" # @param {type:"string"}

# @markdown ---

sampling_steps = 30 # @param {type:"integer"}
CFG_Scale = 7 # @param {type:"number"}
Denoising_strength = 0.5 # @param {type: "number"}

height = 900 # @param {type:"integer"}
width = 673 # @param {type:"integer"}
count = 3 # @param {type:"integer"}
seed = -1 # @param {type:"integer"}
resize_factor = 1.5 # @param {type:"number"}
sampler = "EulerAncestralDiscrete" # @param ['DDIM', 'DDPM', 'DEISMultistep', 'DPMSolverMultistep', 'DPMSolverSinglestep', 'EulerAncestralDiscrete', 'EulerDiscrete', 'HeunDiscrete', 'KDPM2AncestralDiscrete', 'KDPM2Discrete', 'UniPCMultistep']
resize_step_factor = 1.5 # @param {type:"number"}
resize_mode = "latent" # @param ["latent", "2x"]
hires_strength = 0.1 # @param {type:"number"}
clip_skip = 2 # @param {type:"integer"}
controlnet_type = "sketch" # @param ["none", "sketch", "canny", "softedge","pix2pix","bae","pose","i2i"]
controlnet_factor = 0.4 # @param {type:"number"}
controlnet_factor = float(controlnet_factor)
device = "cuda"
generator = torch.Generator(device)

!mkdir /content/save_dir/
!mkdir /content/saved_all/

width = int(width // 64) * 64
height = int(height // 64) * 64

def main():
  global seed,resize_factor, sampling_steps,height, width, sampler,CFG_Scale,Denoising_strength,prompt,gender,fix_hair_color,fix_eyes_color,is_random_hair,is_random_face
  device = "cuda"
  with torch.inference_mode():
    save_dir = "/content/save_dir/"
    local_seed = get_seed(seed)
    delete_all_files(save_dir)
    if controlnet_type != "none":
      print("controlnet_type is not none")
      if controlnet_type == "canny":
        controlnet = ControlNetModel.from_pretrained("lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16)
        con_img = controlnet_canny()
      elif controlnet_type == "sketch":
        hed = HEDdetector.from_pretrained('lllyasviel/Annotators')
        con_img = hed(get_control_target(), scribble=True)
        controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_scribble", torch_dtype=torch.float16)
      elif controlnet_type == "softedge":
        #con_img = controlnet_normal(get_control_target())
        controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_softedge", torch_dtype=torch.float16)
        processor = HEDdetector.from_pretrained('lllyasviel/Annotators')
        processor = PidiNetDetector.from_pretrained('lllyasviel/Annotators')
        con_img = processor(get_control_target(), safe=True)
      elif controlnet_type == "pix2pix":
        con_img = get_control_target()
        controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11e_sd15_ip2p", torch_dtype=torch.float16)
      elif controlnet_type == "bae":
        detecter = NormalBaeDetector.from_pretrained("lllyasviel/Annotators")
        con_img = detecter(get_control_target())
        controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_normalbae", torch_dtype=torch.float16)
      elif controlnet_type == "pose":
        controlnet = ControlNetModel.from_pretrained("lllyasviel/control_v11p_sd15_openpose", torch_dtype=torch.float16)
        posedetecter = OpenposeDetector.from_pretrained('lllyasviel/ControlNet')
        con_img = posedetecter(get_control_target(), hand_and_face=True)
      elif controlnet_type == "i2i":
        con_img = Image.open(get_control_target())

      controlnet.mid_block_scale_factor = controlnet_factor
      con_img = resize_image(con_img, width, height)
      pipe = StableDiffusionControlNetPipeline.from_single_file(
          model_path,
          controlnet=controlnet,
          torch_dtype=torch.float16,
          vae=vae,
          load_safety_checker=False,
          extract_ema=True,
          custom_pipeline="lpw_stable_diffusion",
          clip_skip=2,
          strength=Denoising_strength
      )
      #print(str(**pipe.components))
      i2i_pipe = StableDiffusionImg2ImgPipeline.from_single_file(
          model_path,
          torch_dtype=torch.float16,
          vae=vae,
          load_safety_checker=False,
          extract_ema=True,
          custom_pipeline="lpw_stable_diffusion",
          clip_skip=2,
          strength=hires_strength,
      )
    else:
      print("no controlnet")
      pipe = StableDiffusionPipeline.from_single_file(
          model_path,
          torch_dtype=torch.float16,
          use_safetensors=True,
          vae=vae,
          load_safety_checker=False,
          extract_ema=True,
          custom_pipeline="lpw_stable_diffusion",
          clip_skip=2,
          strength=Denoising_strength
          )
      i2i_pipe = StableDiffusionImg2ImgPipeline(**pipe.components)

    pipe.scheduler = SCHEDULERS_LIST[sampler].from_pretrained(
        pretrained_model_name_or_path="mfrashad/hassakuv13",
        torch_dtype=torch.float16,
        cache_dir="./cache",
        subfolder='scheduler'
    )


    i2i_pipe.scheduler = SCHEDULERS_LIST[sampler].from_pretrained(
        pretrained_model_name_or_path="mfrashad/hassakuv13",
        torch_dtype=torch.float16,
        cache_dir="./cache",
        subfolder='scheduler',
    )
    try:
      pipe = pipe.to("cuda")
      i2i_pipe.to("cuda")
    except Exception as e:
      print(e)

    print("ram reduce.")
    # ram reduce

    #pipe.enable_model_cpu_offload()
    #i2i_pipe.enable_model_cpu_offload()

    pipe.enable_vae_tiling()
    pipe.enable_xformers_memory_efficient_attention()

    #i2i_pipe.enable_vae_tiling()
    i2i_pipe.enable_xformers_memory_efficient_attention()


    # random json reader
    # @markdown ----
    rnd_dir_path = "/content/drive/MyDrive/models/random_list/" # @param {type:"string"}
    rnd_files = get_rnd_json_files(rnd_dir_path)
    print(rnd_files)

    rnd_data = merge_json(rnd_files)
    print(f"Random Face: {str(is_random_face)}, Random Hair: {str(is_random_hair)}")
    if is_random_hair == True:
        if fix_hair_color == "":
            fix_eyes_color = None
        prompt += make_randomized_hair(rnd_data,fix_hair_color)

    if is_random_face == True:
        if fix_eyes_color == "":
            fix_eyes_color = None
        prompt += make_randomized_face(rnd_data,fix_eyes_color,gender)

    if gender != "none":
        add_chara = random.choice(rnd_data[gender])
        prompt += f"({add_chara}),"

    # prompt tokenize
    lora_list = get_loras_from_prompt(prompt)
    loader = lora_loader(pipe, lora_list)
    loader.fusing_loras()
    non_lora_prompt = remove_lora(prompt)
    non_lora_neg = remove_lora(neg)
    set_embs(non_lora_neg, pipe)

    positive_embeds, negative_embeds = get_prompt_with_weight(pipe, non_lora_prompt,non_lora_neg)
    #positive_embeds, negative_embeds = token_auto_concat_embeds(pipe, non_lora_prompt,non_lora_neg)

    #textual_inversion_manager = DiffusersTextualInversionManager(pipe)
    #compel = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder,
    #textual_inversion_manager=textual_inversion_manager,
    #    truncate_long_prompts=False,
    #    device="cuda")

    #positive_embeds = compel(tokenize_line(non_lora_prompt, pipe.tokenizer))
    #negative_embeds = compel(tokenize_line(non_lora_neg, pipe.tokenizer))
    #positive_embeds, negative_embeds = compel.pad_conditioning_tensors_to_same_length([concat_tensor(positive_embeds), concat_tensor(negative_embeds)])
    ##################
    #pipe.enable_xformers_memory_efficient_attention()
    #pipe.enable_model_cpu_offload()
    #i2i_pipe.enable_xformers_memory_efficient_attention()
    #i2i_pipe.enable_model_cpu_offload()
    start = time.time()
    for i in range(0, count, 1):
      local_seed = local_seed + i
      if controlnet_type != "none":
        base_image = gen_control_base_img(pipe, local_seed, positive_embeds, negative_embeds, con_img, lora_list)
      elif controlnet_type == "i2i":
        base_image = con_img
      else:
        base_image = gen_base_img(pipe, local_seed, positive_embeds, negative_embeds,lora_list)
      if resize_mode == "2x":
        width, height = base_image.size
        new_width = int(width * resize_factor)
        new_height = int(height * resize_factor)
        base_image = gen_upscaler_by_diffusers(base_image, local_seed)
        base_image = upscale_image2(base_image, new_width, new_height)
        upscaled_img = gen_upscale_img(i2i_pipe, base_image, 1,local_seed,positive_embeds, negative_embeds)
      else:
        #loader2 = lora_loader(i2i_pipe, lora_list)
        #loader2.fusing_loras()
        upscaled_img = gen_upscale_img(i2i_pipe, base_image, resize_factor,local_seed,positive_embeds, negative_embeds)
      #dif_upscaled_img = gen_upscaler_by_diffusers(base_image,local_seed)
      base_name = str(int(time.time())) + "_" + str(local_seed)
      filename = os.path.join(save_dir,base_name + ".png")
      base_flename = os.path.join(save_dir,base_name + "_base.png")

      allsave_name = os.path.join("/content/saved_all/", base_name + ".png")
      if controlnet_type != "none":
        con_img.save("con_tag.png")
      #diffup_filename = os.path.join(save_dir, base_name + "_difscaled.png")

      #meta save
      meta_size = f"{str(width)}x{str(height)}"
      meta_model_name = os.path.basename(model_path)
      save_meta(upscaled_img, filename,prompt, neg, sampling_steps,
                               sampler, CFG_Scale, local_seed, meta_size, "None", meta_model_name,
                               Denoising_strength, 0)

      #upscaled_img.save(filename)
      #base_image.save(base_flename)

      upscaled_img.save(allsave_name)
      #dif_upscaled_img.save(diffup_filename)
    end = time.time()
    print(f"time elapsed: {end-start}sec")
    #end


def get_seed(seed):
  if seed < 0:
    return random.randrange(0, 4294967295, 1)
  else:
    return seed

def gen_base_img(pipe,seed, positive_embeds, negative_embeds, lora_list):
  global width, height, CFG_Scale, prompt, neg, sampling_steps,clip_skip, generator
  gen_seed = get_seed(seed)
  generator.manual_seed(gen_seed)
  result = pipe(
      prompt_embeds=positive_embeds,
      negative_prompt_embeds=negative_embeds,
      guidance_scale=CFG_Scale,
      width=width,
      height=height,
      generator=generator,
      num_inference_steps=sampling_steps,
      output_type="latent"
    ).images
  with torch.no_grad():
      image = pipe.decode_latents(result)
  image = pipe.numpy_to_pil(image)[0]
  return image

def gen_base_img_i2i(pipe,seed, positive_embeds, negative_embeds, lora_list,i2i_image):
  global width, height, CFG_Scale, prompt, neg, sampling_steps,clip_skip, generator
  gen_seed = get_seed(seed)
  generator.manual_seed(gen_seed)
  result = pipe(
      prompt_embeds=positive_embeds,
      negative_prompt_embeds=negative_embeds,
      guidance_scale=CFG_Scale,
      width=width,
      height=height,
      generator=generator,
      num_inference_steps=sampling_steps,
      output_type="latent",
      image = i2i_image
    ).images
  with torch.no_grad():
      image = pipe.decode_latents(result)
  image = pipe.numpy_to_pil(image)[0]
  return image

def gen_control_base_img(pipe,seed, positive_embeds, negative_embeds, con_img, lora_list):
  global width, height, CFG_Scale, prompt, neg, sampling_steps,clip_skip, generator
  gen_seed = get_seed(seed)
  generator.manual_seed(gen_seed)
  result = pipe(
      prompt_embeds=positive_embeds,
      negative_prompt_embeds=negative_embeds,
      image=con_img,
      guidance_scale=CFG_Scale,
      width=width,
      height=height,
      generator=generator,
      num_inference_steps=sampling_steps,
      output_type="latent"
    ).images
  with torch.no_grad():
      image = pipe.decode_latents(result)
  image = pipe.numpy_to_pil(image)[0]
  return image

def gen_upscale_img(i2i_pipe, img,factor,seed,positive_embeds,negative_embeds):
  global width, height, CFG_Scale, prompt, neg, sampling_steps,clip_skip, generator,resize_step_factor,hires_strength
  resized_image = upscale_image(img, factor)
  resized_image.save("resized.png")
  i2i_step = int(sampling_steps*resize_step_factor)
  img2img = i2i_pipe(
      prompt_embeds=positive_embeds,
      negative_prompt_embeds=negative_embeds,
      image=resized_image,
      num_inference_steps=i2i_step,
      output_type="latent",
      strength=hires_strength,
    ).images
  with torch.no_grad():
      image = i2i_pipe.decode_latents(img2img)
  image = i2i_pipe.numpy_to_pil(image)[0]
  return image

def gen_upscaler_by_diffusers(img, seed):
  upscale_model_id = "stabilityai/sd-x2-latent-upscaler"
  upscaler = StableDiffusionLatentUpscalePipeline.from_pretrained(upscale_model_id, torch_dtype=torch.float16)
  upscaler.to("cuda")

  upscaled_image = upscaler(
      prompt=prompt,
      image=img,
      num_inference_steps=20,
      guidance_scale=0,
      generator=generator,
  ).images[0]
  return upscaled_image

def upscale_image(image, factor, resample=Image.LANCZOS):
    """
    画像を何倍にするかの引数を受け取って、imageのsizeから新しい画像のサイズを計算して小数点を丸めた値でリサイズする

    Args:
        image: リサイズ対象の画像
        factor: リサイズ倍率
        resample: リサンプリング方法

    Returns:
        リサイズ後の画像
    """

    width, height = image.size

    # 新しい画像のサイズを計算
    new_width = int(width * factor)
    new_height = int(height * factor)

    # 小数点を丸めてリサイズ
    image = image.resize((new_width, new_height), resample)
    enhancer = ImageEnhance.Sharpness(image)
    # enhancerオブジェクトの強調
    image = enhancer.enhance(1)
    return image

def upscale_image2(image, width, height, resample=Image.LANCZOS):
    # 小数点を丸めてリサイズ
    image = image.resize((width, height), resample)
    enhancer = ImageEnhance.Sharpness(image)
    # enhancerオブジェクトの強調
    image = enhancer.enhance(1)
    return image

def delete_all_files(dir_path):
    """
    フォルダ内のファイルをすべて削除する

    Args:
        dir_path: フォルダパス
    """
    for file in os.listdir(dir_path):
      try:
        os.remove(os.path.join(dir_path, file))
      except Exception as e:
        print(e)

def make_randomized_face(data,gender,me_color=None,chara=None,chara_num=1):
    prompt = ""

    me = pick_rnd(data["me"])
    if me_color == None:
        me_color = pick_rnd(data["me_color"])
    me_add = pick_rnd(data["me_add"])
    hana = pick_rnd(data["hana"])
    kuchi = pick_rnd(data["kuchi"])
    ago = pick_rnd(data["ago"])

    #has_add  = random.choices({True,False},weights=[0.25,0.75],k=1)[0]

    prompt += set_word(me,1.1)
    if me_color is not None:
        prompt += set_word(me_color)

    prompt += set_word(hana,1.1)
    prompt += set_word(kuchi,1.1)
    prompt += set_word(ago,1.1)

    if chara is not None:
        chara_mix = random.sample(data[chara],chara_num)
        chara_prompt = ""
        for mix in chara_mix:
          chara_prompt += f"({mix}),"
        prompt += chara_prompt
        # prompt += set_word(pick_rnd[data[chara]],1.1)

    return prompt

def make_randomized_hair(data,color=None):
    prompt = ""
    maegami = pick_rnd(data["maegami"])
    usirogami = pick_rnd(data["usirogami"])
    nagasa = pick_rnd(data["nagasa"])
    yokogami = pick_rnd(data["yokogami"])
    hair_type = pick_rnd(data["hair_type"])
    hair_color = pick_rnd(data["hair_color"])

    form = random.choices(["random","fix"],weights=[0.05,0.95],k=1)[0]
    has_yokogami = random.choices([True,False],weights=[0.4,0.6],k=1)[0]
    has_nagasa = random.choices([True,False],weights=[0.3,0.7],k=1)[0]
    has_hair_type = random.choices([True,False],weights=[0.3,0.7],k=1)[0]

    prompt += set_word(maegami,1.1)
    prompt += set_word(usirogami,1.1)
    if has_yokogami:
        prompt += set_word(yokogami,1.1)
    if has_nagasa:
        prompt += set_word(nagasa)
    if has_hair_type:
        prompt += set_word(hair_type)

    if color:
        prompt += f"{color},"
    else:
        prompt += set_word(hair_color)
    print("$$$$ randomized prompt $$$$")
    print(prompt)
    return prompt


def set_word(word,weight=1.0):
    return f"({word}:{str(weight)}),"

def pick_rnd(my_list):
    if not my_list:  # リストが空でないかを確認
        return None
    return random.choice(my_list)

def load_json(filename):
    try:
        with open(filename, 'r', encoding='utf-8') as file:
            data = json.load(file)
            return data
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
    except json.JSONDecodeError:
        print("Error: There was a problem decoding the JSON data.")
    except Exception as e:
        print(f"An error occurred: {e}")

def merge_json(files):
    merged_data = {}

    for file in files:
        temp_data = load_json(file)
        merged_data.update(temp_data)  # 辞書を統合

    return merged_data

def get_rnd_json_files(directory):
    try:
        # 指定されたディレクトリ内の全ファイルを取得
        files = os.listdir(directory)

        # 'rnd_'で始まり、拡張子が'.json'のファイルをフィルタリング
        rnd_json_files = [os.path.join(directory,file) for file in files if file.startswith('rnd_') and file.endswith('.json')]
        return rnd_json_files

    except FileNotFoundError:
        print(f"Error: The directory {directory} was not found.")
    except Exception as e:
        print(f"An error occurred: {e}")


main()
